In [52]:
# %%capture 
# !pip install -U tokenizers
# !pip install -U transformers
# !pip install -U tokenizers
# !pip install -U transformers

In [31]:
import datasets
import sys

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

sys.path.append('../../../')
from transformers.models.bert.tokenization_bert import BertTokenizer
from transformers.models.bert.modeling_linbert import BertForSequenceClassification, BertForMaskedLM

import torch

In [32]:
torch.cuda.device_count()

1

In [33]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, DataCollatorForLanguageModeling


In [34]:

# from transformers_modified.models.bert.tokenization_bert import BertTokenizer
# BertModelForSequenceClassification
import torch
import numpy as np

In [36]:
model = BertForMaskedLM.from_pretrained('M-FAC/bert-tiny-finetuned-sst2')
tokenizer = BertTokenizer.from_pretrained('M-FAC/bert-tiny-finetuned-sst2')
seq_class_model = BertForSequenceClassification.from_pretrained('M-FAC/bert-tiny-finetuned-sst2')

Some weights of BertForMaskedLM were not initialized from the model checkpoint at M-FAC/bert-tiny-finetuned-sst2 and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
for parameter_lm, parameter_cl in zip(model.bert.parameters(), seq_class_model.bert.parameters()):
    print(parameter_lm.data.size() == parameter_cl.data.size())

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [7]:
model.linearize(32, 41)

linearized


In [25]:
def create_data(tokenizer):
    train_set = datasets.load_dataset('sst2', split='train').remove_columns(['idx', 'label'])
    val_set = datasets.load_dataset('sst2', split='validation').remove_columns(['idx', 'label'])

    dynamic_padding = True

    def tokenize_func(examples):
        return tokenizer(examples["sentence"], max_length=32, padding='max_length', truncation=True)

    encoded_dataset_train = train_set.map(tokenize_func, batched=True)
    encoded_dataset_test = val_set.map(tokenize_func, batched=True)
   # data_collator = DataCollatorWithPadding(tokenizer)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15)
    return encoded_dataset_train, encoded_dataset_test, data_collator

def create_mlm_data(tokenizer):
    train_set = datasets.load_dataset('imdb', split='train').remove_columns(['label'])
    val_set = datasets.load_dataset('imdb', split='test').remove_columns(['label'])

    def tokenize_func(examples):
        return tokenizer(examples["text"], max_length=32, padding='max_length', truncation=True)

    encoded_dataset_train = train_set.map(tokenize_func, batched=True)
    encoded_dataset_val = val_set.map(tokenize_func, batched=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer)

    return encoded_dataset_train, encoded_dataset_val, data_collator

In [ ]:
encoded_dataset_train, encoded_dataset_test, data_collator = create_data(tokenizer)

In [26]:
mlm_set, mlm_val_set, mlm_collator = create_mlm_data(tokenizer)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [11]:
metric = datasets.load_metric('accuracy')

def compute_metrics(eval_pred):
	predictions, labels = eval_pred
	predictions = np.argmax(predictions, axis=1)
	return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_679939/2290328016.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('accuracy')


In [28]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    load_best_model_at_end=True,
    num_train_epochs=5,
    weight_decay=0.1,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="epoch",
    seed=42,
    save_strategy = "epoch",
    save_total_limit=5,
    logging_strategy="epoch",
    report_to="all",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mlm_set,
    eval_dataset=mlm_val_set,
    data_collator=mlm_collator,
    #compute_metrics=compute_metrics,
)

In [ ]:
for param in model.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,6.628900,6.589503
2,6.546400,6.511045
3,6.505500,6.479131
4,6.482900,6.447707
5,6.476000,6.435288


TrainOutput(global_step=980, training_loss=6.527945133131378, metrics={'train_runtime': 83.4396, 'train_samples_per_second': 1498.089, 'train_steps_per_second': 11.745, 'total_flos': 10784112000000.0, 'train_loss': 6.527945133131378, 'epoch': 5.0})

In [30]:
trainer.model.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-1): 2 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [17]:
seq_class_model = BertForSequenceClassification.from_pretrained('M-FAC/bert-tiny-finetuned-sst2')

In [18]:
seq_class_model.bert = trainer.model.bert

In [24]:
seq_class_model.bert.encoder.layer[0].attention.self.E

Parameter containing:
tensor([[ 0.0107,  0.0228,  0.1497,  ...,  0.0589,  0.1436,  0.0942],
        [ 0.0464,  0.0290, -0.0227,  ...,  0.0387,  0.1120,  0.0134],
        [-0.0765, -0.1274,  0.1461,  ...,  0.0350,  0.0740,  0.0756],
        ...,
        [ 0.1165, -0.1550, -0.0914,  ..., -0.0647, -0.0303,  0.0779],
        [-0.0241,  0.1494,  0.1148,  ..., -0.0482,  0.1539, -0.0846],
        [-0.0982, -0.0761,  0.0075,  ...,  0.1350, -0.0933,  0.1502]],
       device='cuda:0', requires_grad=True)